In [ ]:
%%capture
!pip install geemap
!pip install retry
!pip install global_land_mask

In [ ]:
import ee
ee.Authenticate()
from global_land_mask import globe
import requests
from retry import retry
import os.path
import shutil
import multiprocessing
import logging
import geemap
import numpy as np
import random
import cv2

In [ ]:
#Delete all downloaded images
for i in range(1,9):
  for file in os.listdir(str(i)):
    os.remove(f"{str(i)}/{file}")

In [ ]:
#Create folders for downloaded images
for i in range(1,9):
  os.mkdir(str(i))

In [ ]:
#Set up parameters
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com")

params = {
    'regions': 100,
    'pointsperregion': 50,
    'regionsize': 0.5,          #Side length of ROI (major) image, in degrees
    'step': 1 / 120,            #1 / 120 is the lowest possible grid size (30 arc seconds). See GPW-4 documentation for other possible grid sizes.
    'seed': 27,
    'dimensions': '224x224',
    'format': 'png',
    'prefix': '',
    'processes': 10,
    'bands': ['B4','B3','B2'],
    'max': 2800,
    'test': False
}

#Define perimeters of areas for which good quality population data exists:
ITALY = ee.Geometry.Polygon([[[7.15, 45.5], [7.15, 41], [14.5, 41], [14.5, 45.5],[7.15,45.5]]], None, False)
SOUTHAFRICA = ee.Geometry.Polygon([[[26,-24],[26,-33.8],[33,-33.8],[33,-24],[26,-24]]], None, False)
AMERICA = ee.Geometry.Polygon([[[-127,49],[-66,49],[-66,30],[-127,30],[-127,49]]], None, False)
BRAZILURU = ee.Geometry.Polygon([[[-60,-3],[-35, -3],[-35,-35],[-60,-35],[-60, -3]]], None, False)
UK = ee.Geometry.Polygon([[[-7.5,56.6],[1,56.6],[1,50.1],[-7.5,50.1],[-7.5,56.6]]], None, False)
AUSTRALIANZ = ee.Geometry.Polygon([[[142,-27],[179,-27],[179,-47],[142,-47],[142,-27]]], None, False)

qualitycountries = (ITALY, SOUTHAFRICA, AMERICA, BRAZILURU, UK, AUSTRALIANZ)

#Manually add some urban areas to the dataset to make it more representative of what users might actually use the model for (London, Paris, Johannesburg, Sao Paolo, Los Angeles, New York City, Milan, Merida (Mexico), Sydney, Mexico City):
CITIES = [[-0.45, 51.691666667], [2.00833333333, 49.066666667], [27.775, -25.9583333333], [-46.866666667, -23.3833333333], [-118.425, 34.1416666667], [-73.216666667, 40.991666667], [8.725, 45.808333333], [-89.908333333, 21.1], [150.775, -33.675], [-99.308333333, 19.658333333]]

def geoJSONize(point):
  return {'geodesic': False, 'type': 'Point', 'coordinates': [point[0], point[1]]}

In [ ]:
#Get randomly selected non-ocean grid corner locations from which to generate each major image
def getCorners(roicorners=list()):
  _buffer = params['step']
  while len(roicorners) < params['regions']:
    x = random.randint(0, 360 // _buffer)
    y = random.randint(0, 360 // _buffer) 
    x, y = x * _buffer, y * _buffer * 0.5
    x, y = -180 + x, -90 + y
    
    if not globe.is_land(y,x):
      continue

    point = ee.Geometry.Point([x,y])
    for country in qualitycountries:
      if country.contains(point).getInfo():
        roicorners.append((x,y))
  return roicorners

In [ ]:
#Get randomly selected non-polar land areas, all corresponding to grid corners from Y dataset
#Generate grid corner locations by stepping by the grid size (buffer*2)
def getImage(corner):
  _size = params['regionsize']
  #Generate square from northeast corner
  roi = ee.Geometry.Polygon([[[corner[0], corner[1]], [corner[0]+_size,corner[1]], [corner[0]+_size, corner[1]-_size], [corner[0],corner[1]-_size], [corner[0], corner[1]]]], None, False)
  #Load image
  image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
  .filterBounds(roi) \
  .filterDate('2020-05-01', '2020-09-01') \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
  .sort('GENERAL_QUALITY') \
  .mosaic() \
  .clip(roi) \
  .select(params['bands'])
  return image


In [ ]:
#Test display image in true colors
corner = corners[0]
image = getImage(corner)
Map = geemap.Map()
roi = ee.Geometry.Polygon([[[corner[0], corner[1]], [corner[0]+0.5,corner[1]], [corner[0]+0.5, corner[1]-0.5], [corner[0],corner[1]-0.5], [corner[0], corner[1]]]], None, False)
Map.addLayer(image, {'bands': params['bands'], 'max': params['max'] + random.randint(-800, 800)}, "Image")
Map.addLayer(roi, {}, "roi", False)
Map.setCenter(image.getInfo()['properties']['system:footprint']['coordinates'][0][0][0] + 0.25, image.getInfo()['properties']['system:footprint']['coordinates'][0][0][1] -0.25)
Map

Map(center=[-24.22083333333333, -59.483333333333334], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
#Generate random center points within image region corresponding to Y dataset grid centers and return them on GeoJSON format
def getPoints(image,corner):
  points = []
  while len(points) < params['pointsperregion']:
    y = corner[0] + (random.randint(1, 0.5 // params['step'])-1) * params['step'] + params['step']/2
    x = corner[1] - (random.randint(1, 0.5 // params['step'])-1) * params['step'] - params['step']/2
    if globe.is_land(x,y):
      points.append((y,x))
  return map(geoJSONize, points)

In [ ]:
#Test GetPoints function:
image = getImage(corners[0])
items = getPoints(image, corners[0])
list(items)

In [ ]:
#Get URLs for minor image download based on center locations and buffer
def getUrls(image, point):
    point = ee.Geometry.Point(point['coordinates'])
    region = point.buffer(params['step']/2, proj='EPSG:4326').bounds()
    try:
      url = image.getThumbURL({
          'region': region,
          'bands': params['bands'],
          'max': params['max'] + random.randint(-400,400),
          'dimensions': params['dimensions'],
          'format': params['format'],
          'gamma': 1.3
      })
    except Exception as e:
      print(e)
      return None

    return url

In [ ]:
#Test getURLs function:
rois = getCorners()
image = getImage(rois[0])
items = getPoints(image, rois[0])
url = getUrls(image, list(items)[0])
url

In [ ]:
#Set up for download. Credit to Qiusheng Wu.
@retry(tries=100, delay=1, backoff=2)
def getRes(point, url):
  if not url:
    print("Skipping corrupted image")

  else:
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status

    #Store and name image files so they're convenient to pair with targets
    def getPath(coords):
      _step = params['step']
      if coords[1] > 0:
        if coords[0] > 90:
          x = (coords[0] - 90) / _step
          y = (90 - coords[1]) / _step  
          return {'directory':'4', 'y': int(y), 'x': int(x)}
        elif 0 < coords[0] < 90:
          x = coords[0] / _step
          y = (90 - coords[1]) / _step  
          return {'directory':'3', 'y': int(y), 'x': int(x)}
        elif -90 < coords[0] < 0:
          x = (coords[0] + 90) / _step
          y = (90 - coords[1]) / _step  
          return {'directory':'2', 'y': int(y), 'x': int(x)}
        elif coords[0] < -90:
          x = (coords[0] + 180) / _step
          y = (90 - coords[1]) / _step  
          return {'directory':'1', 'y': int(y), 'x': int(x)}
      else:
        if coords[0] > 90:
          x = (coords[0] -90) / _step
          y = (coords[1] / _step) * -1 
          return {'directory':'8', 'y': int(y), 'x': int(x)}
        elif 0 < coords[0] < 90:
          x = coords[0] / _step
          y = (coords[1] / _step) * -1 
          return {'directory':'7', 'y': int(y), 'x': int(x)}
        elif -90 < coords[0] < 0:
          x = (coords[0] + 90) / _step
          y = (coords[1] / _step) * -1 
          return {'directory':'6', 'y': int(y), 'x': int(x)}
        elif coords[0] < -90:
          x = (coords[0] + 180) / _step
          y = (coords[1] / _step) * -1 
          return {'directory':'5', 'y': int(y), 'x': int(x)}

    ext = params['format']
    out_dir = os.path.join(params['out_dir'], getPath(point['coordinates'])['directory'])
    basename = f"x{str(getPath(point['coordinates'])['x']).zfill(5)}y{str(getPath(point['coordinates'])['y']).zfill(5)}"
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as outfile:
        shutil.copyfileobj(r.raw, outfile)
    print(f"{basename} downloaded.\n") #Location: {point['coordinates']}\n")

In [ ]:
#Main: get download URLs and coordinates, download images
rois = getCorners()
if params['test'] == False:
  rois.extend(CITIES)
num = 0
for roi in rois:
  num += 1
  print(f"New region: {num}. {roi}")
  image = getImage(roi)
  items = tuple(getPoints(image, roi))
  images = [image] * len(items)
  urls = map(getUrls, images, items)
  iter = zip(items, urls)

  logging.basicConfig()
  pool = multiprocessing.Pool(params['processes'])
  pool.starmap(getRes, iter)
  pool.close()


In [1]:
%%capture
!pip install pytorch
!pip install torch-lr-finder

In [2]:
#Moving on to model setup stage, import PyTorch tools
import numpy as np
import cv2
import torch
import torchvision.transforms as TF
import os.path

In [ ]:
#Load population grids into arrays and pair up saved images with population counts
prefix = "Population"
ext = ".asc"
path = "drive/MyDrive/PopulationData/gpw_v4_population_count_rev11_2020_30_sec_"
dataset = []
for i in range(1,9):
  array = np.loadtxt(f"{path}{i}{ext}", skiprows=6)
  for filename in sorted(os.listdir(str(i))):
    if filename[12:16] != ".png":
      continue

    column = int(filename[1:6])
    row = int(filename[7:12])
    depvar = array[row,column]

    #Convert image to array and reshape array according to the model we will be using
    image = cv2.imread(f"{str(i)}/{filename}", cv2.IMREAD_COLOR)
    del filename
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    #Normalize the image according to the standard for ResNet, which we will be using for transfer learning
    normalize = TF.Compose([
        TF.ToTensor(),
        TF.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    normalizedimage = normalize(image)
    del image

    dataset.append((normalizedimage, torch.from_numpy(np.array(depvar))))
  del array

#Save dataset to file
torch.save(dataset, "drive/MyDrive/PopulationData/dataset.t")

len(dataset)

In [3]:
#Load dataset
dataset = torch.load("drive/MyDrive/PopulationData/dataset.t")
len(dataset)

5361

In [4]:
import torch.nn.functional as F
import torchvision.models as M
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch_lr_finder import LRFinder

/usr/local/lib/python3.10/dist-packages/torch_lr_finder/lr_finder.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
#Hyperparameters:
hyperparameters = {
    'state': 'state0',               #This string is added to the filename of the model when saving to indicate whether the model is pure pretrained (state0), output layer fine-tuned (state1), intermediate state (state2) or fully fine-tuned (state3)
    'train_pct': 0.8,
    'epochs': 6,
    'unfrozen': ['fc'],
    'learning_rate': 0.00001,
    'batch_size': 64,
    'scheduler_max_lr': 0.01,
    'scheduler_div_factor': 1000

}
transformparams = {
    'saturation': 0.5,
    'hue': 0.1,
    'brightness': 0.5,
    'contrast': 0.2,
}

#Set up dataset and augmentations. First initialize randomizer for random transforms.
torch.manual_seed(42)

#List of transformations to apply. Selecting only those that don't affect the outline of the image, as that would bring it out of sync with the population data. Lighting transformations take place inside SpaceTfm because they are applied to the pixel values as log values in order to avoid distortion.
transforms = TF.Compose([
            TF.RandomRotation([90,90]),
            TF.RandomHorizontalFlip(),
            TF.RandomVerticalFlip(),
            TF.ColorJitter(brightness=(1-transformparams['brightness'],1+transformparams['brightness']),
                           contrast=(1-transformparams['contrast'],1+transformparams['contrast']),
                           saturation=(1-transformparams['saturation'],1+transformparams['saturation']),
                           hue=(-transformparams['hue'],transformparams['hue']))
])

#Split the dataset randomly into a training set and a validation set
train, valid = random_split(dataset, [hyperparameters['train_pct'],1-hyperparameters['train_pct']])

#Define a custom dataset class to accommodate applying transformations to my already collated dataset (transformations are normally
#done before the image is converted to tensor and paired up with its target value)
class CustomDataset(Dataset):
    def __init__(self, data, transforms=None):
        self.data = data
        self.transforms = transforms
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image, target = self.data[idx]
        if self.transforms:
            image = self.transforms(image)
        return image, target

#Instantiate the custom dataset and store the data as a DataLoaders object, which is an iterator that provides the data one batch at a time. 
#The training dataset is initialized with transforms, whereas the validation dataset is initialized without transforms. 
#The data is now ready to be input into the model.
tds = CustomDataset(train, transforms=transforms)
vds = CustomDataset(valid)
tdl = DataLoader(dataset=tds, batch_size=hyperparameters['batch_size'], shuffle=True)
vdl = DataLoader(dataset=vds, batch_size=hyperparameters['batch_size'], shuffle=True)


In [ ]:
len(tdl)

In [6]:
#Import pretrained model for transfer learning
model = M.resnet18(weights='ResNet18_Weights.DEFAULT')

#Add custom head to model to get the right number of outputs and add a little extra power
customhead = nn.Sequential(
    nn.Linear(512,1),
)

model.fc = customhead

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 197MB/s]


In [45]:
#Set up model

#Specify device to use for training:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

#Freeze the parameters of the earliest convolutional layers so the model doesn't have to relearn basic
#vision capabilities such as detecting edges, corners, squares, circles etc. Since my imported model was
#trained on a much larger dataset and using far more computations than I can, it's more efficient from me
#to borrow the basic capabilities from the imported model. Even though the images in my dataset are very
#different, being satellite images, they still have edges, corners and polygons.
for name, param in model.named_parameters():
  for layername in hyperparameters['unfrozen']:
    if layername in name:
      param.requires_grad = True
    else:
      param.requires_grad = False

#Create Root Mean Squared Log Error loss function. I'm using RMSLE as opposed RMSE
#because I don't want error scale to matter. If my model guesses 9000 people living in an area with 8000
#people, that is as good a guess as guessing 90 people in an area with 80 people.
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred, targ):
        #return torch.sqrt(self.mse(torch.log(pred), torch.log(targ)))
        return torch.sqrt(torch.mean(torch.log(abs(pred+1/targ+1))))

criterion = RMSLELoss()
optimizer = optim.Adam(model.parameters(), lr=hyperparameters['learning_rate'])
#scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hyperparameters['scheduler_max_lr'], steps_per_epoch=len(tdl), epochs=hyperparameters['epochs'], div_factor=hyperparameters['scheduler_div_factor'])

#Initialize TensorBoard SummaryWriter for more in-depth metrics and diagnostics:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('/')

In [41]:
testx = np.array([20,20,20,20,20])
testx2 = np.array([2,2,2,2,2])
testx = torch.from_numpy(testx)
testx2 = torch.from_numpy(testx2)
testy = np.array([10,10,10,10,10])
testy2 = np.array([1,1,1,1,1])
testy = torch.from_numpy(testy)
testy2 = torch.from_numpy(testy2)

loss = RMSLELoss()
loss.forward(testx, testy), loss.forward(testx2, testy2)



(tensor(1.0986), tensor(1.0986))

In [43]:
#Define evaluation function for model
def accuracy(valid):
  
  eval = {}

  #Calculate total validation loss:
  eval['validloss'] = 0.0
  eval['metrics'] = {}
  eval['metrics']['c_50'] = 0
  eval['metrics']['c_mag'] = 0
  eval['metrics']['c_10'] = 0

  for x, y in valid:
    x = x.to(device)
    y = y.to(device)
    preds = model(x)
    loss = criterion(preds, y)
    eval['validloss'] += loss

    #Calculate precision metrics: 
    for prediction, target in zip(preds.flatten(), y.flatten()):
      if abs(prediction - target).item() < target.item():
        eval['metrics']['c_50'] += 1
      if target.item() / 10 < prediction.item() < target.item() * 10:
        eval['metrics']['c_mag'] += 1
      if target.item() * 0.9 < prediction.item() < target.item() * 1.1:
        eval['metrics']['c_10'] += 1
  return eval

In [ ]:
#Find the optimal learning rate:
optimizer = optim.Adam(model.parameters(), lr=hyperparameters['learning_rate'])
lr_finder = LRFinder(model, optimizer, criterion, device=device)
lr_finder.range_test(tdl, end_lr=1, num_iter=100)
lr_finder.plot()
lr_finder.reset()

In [46]:
#Fine-tune the model to our dataset
for epoch in range(hyperparameters['epochs']):
  running_loss = 0.0
  batchcount = 0
  for x, y in tdl:
    x = x.to(device)
    y = y.to(device)
    batchcount += 1
    optimizer.zero_grad()
    preds = model(x)
    loss = criterion(preds, y)
    #writer.add_scalar('Loss/train', loss, batchcount)
    #writer.add_histogram('Weights', model.fc.weight)
    loss.backward()
    optimizer.step()
    running_loss += loss

  #Print statistics after each epoch
  eval = accuracy(vdl)
  print(f"Epoch {epoch + 1} finished.")
  print(f"Training loss: {running_loss:.4f}")
  print(f"Validation loss: {eval['validloss']:.4f}")
  print(f"Perfect: {eval['metrics']['c_10'] / len(vdl.dataset) * 100:.2f}%")
  print(f"Correct half to double: {eval['metrics']['c_50'] / len(vdl.dataset) * 100:.2f}%")
  print(f"Correct order of magnitude: {eval['metrics']['c_mag'] / len(vdl.dataset) * 100:.2f}%\n")

Epoch 1 finished.
Training loss: nan
Validation loss: nan
Perfect: 0.00%
Correct half to double: 0.00%
Correct order of magnitude: 0.00%

Epoch 2 finished.
Training loss: nan
Validation loss: nan
Perfect: 0.00%
Correct half to double: 0.00%
Correct order of magnitude: 0.00%

Epoch 3 finished.
Training loss: nan
Validation loss: nan
Perfect: 0.00%
Correct half to double: 0.00%
Correct order of magnitude: 0.00%

Epoch 4 finished.
Training loss: nan
Validation loss: nan
Perfect: 0.00%
Correct half to double: 0.00%
Correct order of magnitude: 0.00%

Epoch 5 finished.
Training loss: nan
Validation loss: nan
Perfect: 0.00%
Correct half to double: 0.00%
Correct order of magnitude: 0.00%

Epoch 6 finished.
Training loss: nan
Validation loss: nan
Perfect: 0.00%
Correct half to double: 0.00%
Correct order of magnitude: 0.00%



In [ ]:
state = hyperparameters['state']
torch.save(model, f'drive/MyDrive/PopulationData/model{state}.pkl')